In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [9]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [10]:
ls -la

total 54
drwx------ 2 root root  4096 Feb 13 13:22 data/
-rw------- 1 root root 35547 Feb 13 18:41 Day4_model.ipynb
-rw------- 1 root root  5004 Feb 14 07:35 Day5.ipynb
drwx------ 2 root root  4096 Feb 11 09:53 .git/
-rw------- 1 root root     5 Feb 13 13:22 .gitignore
-rw------- 1 root root   799 Feb 11 09:50 HelloGithub.ipynb
drwx------ 2 root root  4096 Feb 12 08:45 matrix_one/
-rw------- 1 root root    40 Feb 11 09:53 README.md


In [11]:
ls -la data

total 49444
-rw------- 1 root root 50630230 Feb 13 13:28 men_shoes.csv


In [12]:
df=pd.read_csv("data/men_shoes.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def run_model(features,model=DecisionTreeRegressor(max_depth=5)):
  X=df[features].values
  y=df['prices_amountmin'].values
  scores=cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [0]:
df['brand_cat']=df['brand'].factorize()[0]

In [17]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [22]:
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'],model)

(-57.47223572384038, 4.328288468270897)

In [23]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [24]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  if str(x)=='nan': return []
  return literal_eval(x.replace('\\"','"'))


In [0]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan': return output_dict
  features=literal_eval(x.replace('\\"','"'))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()
    output_dict[key]=value
  return output_dict


In [0]:
df['features_parsed']=df['features'].map(parse_features)

In [55]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
keys=set()

In [60]:
df['features_parsed'].map(lambda x:keys.update(x.keys()))
len(keys)

476

In [61]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [62]:
df[ df['feat_model'].isnull() ].shape

(13951, 526)

In [63]:
df[False == df['feat_model'].isnull() ].shape[0] / df.shape[0] * 100

23.681619256017505

In [0]:
keys_dict = {}
for key in keys:
  keys_dict[key] = df[False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [66]:
{k:v for k,v in keys_dict.items() if v > 10}

{'age group': 27.64770240700219,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'brand': 48.62691466083151,
 'casual & dress shoe style': 12.910284463894966,
 'color': 47.784463894967175,
 'condition': 18.12910284463895,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838,
 'model': 23.681619256017505,
 'shoe category': 24.376367614879648,
 'shoe size': 23.326039387308533,
 'shoe width': 15.74945295404814,
 'size': 21.422319474835884,
 'style': 14.923413566739608}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_model_cat'] = df['feat_model'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_size_cat'] = df['feat_size'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_condition_cat'] = df['feat_condition'].factorize()[0]

In [0]:
for key in keys:
  df[ get_name_feat(key) + '_cat' ] = df[ get_name_feat(key)].factorize()[0]

In [69]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_color/finish family_cat', 'feat_sports league_cat',
       'feat_product id_cat', 'feat_casing_cat', 'feat_number of pockets_cat',
       'feat_sizearm_cat', 'feat_light transmission (vlt)_cat',
       'feat_mirrored_cat', 'feat_combined shipping_cat',
       'feat_frame type_cat'],
      dtype='object', length=1002)

In [70]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].shape

(8846, 2)

In [0]:
feats = ['']

In [72]:
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [73]:
feats_cat = [x for x in df.columns if 'cat' in x]
print(feats_cat)

['categories', 'brand_cat', 'feat_recommended location', 'feat_clothing category', 'feat_fabrication', 'feat_multi pack indicator', 'feat_certifications and listings', 'feat_catalog', 'feat_location - city/state', 'feat_location - country', 'feat_shoe category', 'feat_brand_cat', 'feat_color_cat', 'feat_model_cat', 'feat_gender_cat', 'feat_size_cat', 'feat_sport_cat', 'feat_condition_cat', 'feat_rise_cat', 'feat_enhanced visibility (reflective)_cat', 'feat_product name_cat', 'feat_garment care_cat', 'feat_recommended location_cat', 'feat_fishing_cat', 'feat_safety features_cat', 'feat_part type_cat', 'feat_nickel free_cat', 'feat_accessory type_cat', 'feat_batteries included?_cat', 'feat_country//region of manufacture_cat', 'feat_heat zones_cat', 'feat_is wheeled_cat', 'feat_fit_cat', 'feat_has paper wood_cat', 'feat_fabric_cat', 'feat_case diameter_cat', 'feat_general warranty_cat', 'feat_sub style_cat', 'feat_gold karat_cat', 'feat_color family_cat', 'feat_article_cat', 'feat_clothin

In [0]:
feats = ['brand_cat', 'feat_gender_cat', 'feat_color_cat', 'feat_brand_cat', 'feat_sport_cat', 'feat_condition_cat']

In [0]:
feats += ['feat_case back_cat',
 'feat_fabric type_cat',
 'feat_dimensions_cat',
 'feat_movement_cat',
 'feat_name_cat',
 'feat_number of items_cat',
 'feat_hat style_cat',
 'feat_shoe fastening type_cat',
 'feat_heart rate monitor_cat',
 'feat_case thickness_cat',
 'feat_construction_cat',
 'feat_thick, warm, cozy_cat',
 'feat_manufacturer part number_cat',
 'feat_shipping to usa_cat',
 'feat_polarized_cat',
 'feat_neckline_cat',
 'feat_is energy star-certified_cat',
 'feat_country of origin components:_cat',
 'feat_type 2_cat',
 'feat_seller inventory number_cat',
 'feat_attachment_cat',
 'feat_has paper wood_cat',
 'feat_date_cat',
 'feat_type_cat',
 'feat_place of origin_cat',
 'feat_fabric content_cat',
 'feat_occasion_cat',
 'feat_sole_cat',
 'feat_style name_cat',
 'feat_picture_cat',
 'feat_military style_cat',
 'feat_charger included_cat',
 'feat_leg_cat',
 'feat_eye size:_cat',
 'feat_stone type_cat',
 'feat_temple length_cat',
 'feat_bracelet style_cat',
 'feat_sleeve style_cat',
 'feat_inseam_cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_waist size_cat',
 'feat_lens tint_cat',
 'feat_pocket_cat',
 'feat_primary shelf id_cat',
 'feat_bridge/temple size:_cat',
 'feat_antiscratch lens coating_cat',
 'feat_airport friendly_cat',
 'feat_hardsided or softsided_cat',
 'feat_parts_cat',
 'feat_watch shape_cat',
 'feat_shirt size_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_item color_cat',
 'feat_protects against_cat']


In [93]:
model = RandomForestRegressor(max_depth=5, n_estimators=200,random_state=0)
result = run_model(feats, model)
print(result)

(-58.28135834107793, 4.695741387212425)


In [94]:
X = df[feats].values
y = df['prices_amountmin'].values
model.fit(X,y)
perm = PermutationImportance(model, random_state=0).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2080 ± 0.0107,brand_cat
0.1100 ± 0.0154,feat_color_cat
0.0273 ± 0.0019,feat_gender_cat
0.0218 ± 0.0046,feat_condition_cat
0.0131 ± 0.0013,feat_brand_cat
0.0030 ± 0.0008,feat_case thickness_cat
0.0029 ± 0.0010,feat_case thickness_cat
0.0016 ± 0.0001,feat_dimensions_cat
0.0015 ± 0.0002,feat_manufacturer part number_cat
0.0013 ± 0.0001,feat_fabric content_cat


In [95]:
df['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
masimo capori     0.000055
alpha tactical    0.000055
anarchy           0.000055
jaald             0.000055
poc               0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
!git config --global user.email "djkormo@gmail.com"
!git config --global user.name "djkormo"

In [0]:
!git 

In [0]:
!git add Day5.ipynb

In [90]:
!git commit  -m "Adding Day5.ipynb"

[master fd7883c] Adding Day5.ipynb
 1 file changed, 1 insertion(+)
 create mode 100644 Day5.ipynb


In [91]:
!git push -u origin master

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 12.23 KiB | 3.06 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/djkormo/dw_matrix.git
   8a2e2fb..fd7883c  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
